In [7]:
import threading
import time
import random


# 1. Создание двух классов, с приватными даными(__Х__) и доступом к ним. 
class All():
    def __init__(self, value=0):
        self.__value = value

    def new_1(self):
        return self.__value

    def new_2(self, value):
        self.__value = value

#2. Создать по экземпляру
class One(All):
    pass


class Two(All):
    pass


# Добавление рандомного числа к классам
def add(number):
    value = number.new_1()
    value += random.random()
    number.new_2(value)


# 4. Первые потоки генерируют число и добавляют сначала к 1, потом 2
def add_in_1(first, second, times, lock):
    for _ in range(times):
        lock.acquire()
        add(first)
        add(second)
        lock.release()


# 5. Первые потоки генерируют число и добавляют сначала к 2, потом 1
def add_in_2(first, second, times, lock):
    for _ in range(times):
        lock.acquire()
        add(second)
        add(first)
        lock.release()


# 6. Таймер
def timer(func, *args, **kwargs):
    start = time.perf_counter()
    func(*args, **kwargs)
    end = time.perf_counter()
    print(f'Work Time: {end-start}s')

# Генерация количества чисел для N, K1, K2
def generate_numbers():
    N = random.randint(10, 20)
    K1 = random.randint(10000, 20000)
    K2 = random.randint(10000, 20000)
    return N, K1, K2

#Создание потоков, прировнение к ним рандомного числа, и добавление рандомного числа от 1 или 2 потока
def create_threads(first_digit, second_digit, lock):
    threads = []
    N, K1, K2 = generate_numbers()
    print(f'N = {N}\nK1 = {K1}\nK2 = {K2}')
    for _ in range(N//2):
        thread = threading.Thread(
            target=add_in_1, args=(first_digit, second_digit, K1, lock))
        threads.append(thread)
        thread.start()
    for _ in range(N//2, N):
        thread = threading.Thread(
            target=add_in_2, args=(first_digit, second_digit, K2, lock))
        threads.append(thread)
        thread.start()
    return threads


def main():
    flow_1 = One()
    flow_2 = Two()
    lock = threading.Lock()
    threads = create_threads(flow_1, flow_2, lock)

    # Wait until all threads have finished executing
    for thread in threads:
        thread.join()

    # First and Second values are close together because they both get incremented K1*N/2+K2*N/2 times
    print(
        f'First Value: {flow_1.new_1()}\nSecond Value: {flow_2.new_1()}')


if __name__ == "__main__":
    timer(main)


N = 19
K1 = 14446
K2 = 19105
First Value: 160945.6697778809
Second Value: 160375.85069856496
Work Time: 0.954247400019085s
